### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_confidence = fusion_output_df['confidence'].values

In [3]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [4]:
class KalmanFilter:
    def __init__(self, process_variance=1e-3, measurement_variance=1e-2, initial_estimate=0.5):
        self.x = initial_estimate       
        self.P = 1.0                    
        self.Q = process_variance       
        self.R = measurement_variance   

    def update(self, measurement):
        self.P += self.Q
        K = self.P / (self.P + self.R)
        self.x += K * (measurement - self.x)
        self.P *= (1 - K)
        return self.x

In [5]:
def apply_kalman(confidences):
    kf = KalmanFilter()
    smoothed = []
    for z in confidences:
        smoothed.append(kf.update(z))
    return np.array(smoothed)

### Apply

In [6]:
conf = apply_kalman(fusion_output_confidence)

In [7]:
tau = 0.5
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [8]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-65    3.5   63     0.67   1        Bom      0.92      
-34    2.8   30     0.38   1        Bom      0.92      
-62    7.2   37     0.81   1        Bom      0.92      
-52    1.8   9      0.15   1        Bom      0.92      
-31    2.1   33     0.11   1        Bom      0.92      
-61    3.5   21     0.01   1        Bom      0.93      
-45    3.3   19     0.20   1        Bom      0.93      
-64    7.7   71     0.54   1        Bom      0.94      
-53    3.3   40     0.32   1        Bom      0.92      
-39    2.0   10     0.04   1        Bom      0.92      
-63    2.7   3      0.39   1        Bom      0.92      
-87    3.5   50     0.54   1        Bom      0.92      
-56    7.2   37     0.37   1        Bom      0.91      
-89    8.3   83     0.59   1        Bom      0.92      
-55    4.5   91     0.47   1        Bom      0.93      
-32    2.9   21     0.15   1        Bom      0.9

### Save

In [9]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
